# PyTorch Image Models

GitHub: https://github.com/huggingface/pytorch-image-models

Doc: https://huggingface.co/docs/hub/timm

#### Installation

In [ ]:
!pip install timm

In [ ]:
import timm
import torch
import torch.nn as nn

#### Statistics

In [ ]:
len(timm.list_models("*"))

In [ ]:
len(timm.list_models("*", pretrained=True))

In [ ]:
timm.list_models("res*", pretrained=True)

#### Usage

In [ ]:
MODEL_NAME = "resnet50"

In [ ]:
model = timm.create_model(
    MODEL_NAME,
    pretrained=True)

In [ ]:
model

In [ ]:
model.default_cfg

In [ ]:
# inference
inputs = torch.rand(1, 3, 224, 224)
outputs = model(inputs)
outputs.shape

In [ ]:
# classifier head
model.fc, model.get_classifier()

Custom num_classes

In [ ]:
# get a pretrained model with custom classifier
model = timm.create_model(MODEL_NAME,
    pretrained=True,
    num_classes=3)

In [ ]:
# classifier head
model.get_classifier()

Without classifier: embeddings only

In [ ]:
# get a pretrained model without classifier
model = timm.create_model(MODEL_NAME,
    pretrained=True,
    num_classes=0)

model.get_classifier()

In [ ]:
inputs = torch.rand(1, 3, 224, 224)
outputs = model(inputs)
outputs.shape

#### Custom Classifier

In [ ]:
# Get the in_features of classifier (out_feature of encoder)
model = timm.create_model(MODEL_NAME,
    pretrained=True)

num_features = model.get_classifier().in_features
num_features

In [ ]:
# replace with custom classifier
model.fc = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(num_features, 64),
    nn.LayerNorm(64),
    nn.ReLU(),
    nn.Linear(64, 10),
)

In [ ]:
inputs = torch.rand(1, 3, 224, 224)
outputs = model(inputs)
outputs.shape

#### Feature Extraction

In [ ]:
# get last layer of features

inputs = torch.rand(1, 3, 224, 224)
outputs = model.forward_features(inputs)
outputs.shape # (N, cout, H, W)

multi-stage features (not supported in all models)

In [ ]:
model = timm.create_model(MODEL_NAME,
    pretrained=True,
    features_only=True)

In [ ]:
inputs = torch.rand(1, 3, 224, 224)
outputs = model(inputs)

# multi-stage features
for o in outputs:
    print(o.shape)

#### Image Transform

In [ ]:
# basic transform
timm.data.create_transform((3, 224, 224))

In [ ]:
# Transform from pretrained model
model = timm.create_model(MODEL_NAME,
    pretrained=True)
model.pretrained_cfg

In [ ]:
# Data config only
timm.data.resolve_data_config(model.pretrained_cfg)

In [ ]:
# create transform from pretrained model
data_cfg = timm.data.resolve_data_config(model.pretrained_cfg)
transform = timm.data.create_transform(**data_cfg)
transform

#### Use pretrained model to predict

In [ ]:
import requests
from PIL import Image
from io import BytesIO
url = 'https://datasets-server.huggingface.co/assets/imagenet-1k/--/default/test/12/image/image.jpg'
image = Image.open(requests.get(url, stream=True).raw)
image

In [ ]:
# Create model and data transforms
model = timm.create_model(MODEL_NAME,
    pretrained=True).eval()
transform = timm.data.create_transform(
    **timm.data.resolve_data_config(model.pretrained_cfg)
)

In [ ]:
# transform image to tensor
image_tensor = transform(image)
image_tensor.shape

In [ ]:
# inference
output = model(image_tensor.unsqueeze(0))
output.shape

In [ ]:
# logits to probabilities
probabilities = torch.nn.functional.softmax(output[0], dim=0)
probabilities.shape

In [ ]:
# get top 5 probalities indices
values, indices = torch.topk(probabilities, 5)
indices

In [ ]:
# get imagenet 1000 class names
# mapping class idxs to names
IMAGENET_1k_URL = 'https://storage.googleapis.com/bit_models/ilsvrc2012_wordnet_lemmas.txt'
IMAGENET_1k_LABELS = requests.get(IMAGENET_1k_URL).text.strip().split('\n')
[{'label': IMAGENET_1k_LABELS[idx], 'value': val.item()} for val, idx in zip(values, indices)]